# Step 3: Statistical Analysis and Summary

This notebook provides comprehensive statistical summaries of the AF cohort including:
- Cohort characteristics and demographics
- AF episode patterns and duration
- Medication usage analysis
- Outcomes analysis
- Temporal trends

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sys
sys.path.append('..')
from config import *

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

## Load Cohort Data

In [ ]:
# Load the complete cohort
af_cohort = pd.read_csv(f'../{COHORT_OUTPUT_PATH}/af_cohort_complete.csv', 
                        parse_dates=['af_start', 'af_end', 'icu_intime', 'icu_outtime'])

# Load first episodes
first_episodes = pd.read_csv(f'../{COHORT_OUTPUT_PATH}/af_cohort_first_episodes.csv',
                             parse_dates=['af_start', 'af_end', 'icu_intime', 'icu_outtime'])

# Load patient-level
patient_level = pd.read_csv(f'../{COHORT_OUTPUT_PATH}/af_cohort_patient_level.csv',
                            parse_dates=['af_start', 'af_end', 'icu_intime', 'icu_outtime'])

print(f"Loaded {len(af_cohort):,} AF episodes")
print(f"Loaded {len(first_episodes):,} first episodes")
print(f"Loaded {len(patient_level):,} unique patients")

## 1. Cohort Flow Diagram Data

In [ ]:
print("COHORT SELECTION FLOW")
print("=" * 60)
print(f"Total AF episodes identified: {len(af_cohort):,}")
print(f"  → Unique ICU stays with AF: {af_cohort['stay_id'].nunique():,}")
print(f"  → Unique hospital admissions: {af_cohort['hadm_id'].nunique():,}")
print(f"  → Unique patients: {af_cohort['subject_id'].nunique():,}")
print(f"\nFirst AF episodes per stay: {first_episodes.shape[0]:,}")
print(f"Patients with multiple episodes: {(af_cohort.groupby('stay_id')['episode_number'].max() > 1).sum():,}")

## 2. Demographics (Table 1)

In [ ]:
def create_table1(df, title="Patient Characteristics"):
    """Create a Table 1 style summary"""
    print(f"\n{title}")
    print("=" * 60)
    print(f"N = {len(df):,}\n")
    
    # Age
    print(f"Age (years), median [IQR]: {df['age'].median():.0f} [{df['age'].quantile(0.25):.0f}-{df['age'].quantile(0.75):.0f}]")
    
    # Gender
    print(f"\nGender, n (%)")
    for gender, count in df['gender'].value_counts().items():
        print(f"  {gender}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Race
    print(f"\nRace, n (%)")
    for race, count in df['race'].value_counts().head(5).items():
        print(f"  {race}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Admission type
    print(f"\nAdmission Type, n (%)")
    for atype, count in df['admission_type'].value_counts().items():
        print(f"  {atype}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # ICU type
    print(f"\nFirst ICU Care Unit, n (%)")
    for icu, count in df['first_careunit'].value_counts().head(5).items():
        print(f"  {icu}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Severity
    print(f"\nSeverity Scores")
    if 'sofa_24hours' in df.columns:
        sofa_available = df['sofa_24hours'].notna()
        print(f"  SOFA score (first 24h), median [IQR]: {df.loc[sofa_available, 'sofa_24hours'].median():.0f} [{df.loc[sofa_available, 'sofa_24hours'].quantile(0.25):.0f}-{df.loc[sofa_available, 'sofa_24hours'].quantile(0.75):.0f}]")
        print(f"  (Available for {sofa_available.sum():,} patients, {sofa_available.sum()/len(df)*100:.1f}%)")
    
    return df

# Create Table 1 for patient-level data
create_table1(patient_level, "Table 1: Patient Characteristics (N = unique patients)")

## 3. AF Episode Characteristics

In [ ]:
print("\nAF EPISODE CHARACTERISTICS")
print("=" * 60)

# Episodes per stay
episodes_per_stay = af_cohort.groupby('stay_id')['episode_number'].max()
print(f"\nEpisodes per ICU stay:")
print(f"  Median: {episodes_per_stay.median():.0f}")
print(f"  Mean (SD): {episodes_per_stay.mean():.1f} ({episodes_per_stay.std():.1f})")
print(f"  Range: {episodes_per_stay.min():.0f} - {episodes_per_stay.max():.0f}")

# AF duration
print(f"\nAF Episode Duration (hours):")
print(f"  Median [IQR]: {af_cohort['af_hours'].median():.1f} [{af_cohort['af_hours'].quantile(0.25):.1f}-{af_cohort['af_hours'].quantile(0.75):.1f}]")
print(f"  Mean (SD): {af_cohort['af_hours'].mean():.1f} ({af_cohort['af_hours'].std():.1f})")

# Duration categories
print(f"\nAF Duration Categories:")
af_cohort['duration_category'] = pd.cut(af_cohort['af_hours'], 
                                        bins=[0, 1, 6, 24, 48, np.inf],
                                        labels=['<1h', '1-6h', '6-24h', '24-48h', '>48h'])
for cat, count in af_cohort['duration_category'].value_counts().sort_index().items():
    print(f"  {cat}: {count:,} ({count/len(af_cohort)*100:.1f}%)")

# Timing relative to ICU admission
af_cohort['hours_from_icu_admit'] = (af_cohort['af_start'] - af_cohort['icu_intime']).dt.total_seconds() / 3600
print(f"\nTime from ICU admission to AF start (hours):")
print(f"  Median [IQR]: {af_cohort['hours_from_icu_admit'].median():.1f} [{af_cohort['hours_from_icu_admit'].quantile(0.25):.1f}-{af_cohort['hours_from_icu_admit'].quantile(0.75):.1f}]")

## 4. Medication Usage Analysis

In [ ]:
print("\nMEDICATION USAGE DURING AF EPISODES")
print("=" * 60)

# Episode-level medication usage
print(f"\nEpisode-level (N={len(af_cohort):,}):")
print(f"  Received antiarrhythmic: {af_cohort['received_antiarrhythmic'].sum():,} ({af_cohort['received_antiarrhythmic'].mean()*100:.1f}%)")
print(f"  Received rate control: {af_cohort['received_rate_control'].sum():,} ({af_cohort['received_rate_control'].mean()*100:.1f}%)")

# Medication combinations
print(f"\nMedication Combinations:")
both = (af_cohort['received_antiarrhythmic']) & (af_cohort['received_rate_control'])
aa_only = (af_cohort['received_antiarrhythmic']) & (~af_cohort['received_rate_control'])
rate_only = (~af_cohort['received_antiarrhythmic']) & (af_cohort['received_rate_control'])
neither = (~af_cohort['received_antiarrhythmic']) & (~af_cohort['received_rate_control'])

print(f"  Both AA and Rate Control: {both.sum():,} ({both.mean()*100:.1f}%)")
print(f"  Antiarrhythmic only: {aa_only.sum():,} ({aa_only.mean()*100:.1f}%)")
print(f"  Rate control only: {rate_only.sum():,} ({rate_only.mean()*100:.1f}%)")
print(f"  Neither: {neither.sum():,} ({neither.mean()*100:.1f}%)")

# Patient-level
print(f"\nPatient-level (N={len(patient_level):,}):")
print(f"  Ever received antiarrhythmic: {patient_level['received_antiarrhythmic'].sum():,} ({patient_level['received_antiarrhythmic'].mean()*100:.1f}%)")
print(f"  Ever received rate control: {patient_level['received_rate_control'].sum():,} ({patient_level['received_rate_control'].mean()*100:.1f}%)")

## 5. Outcomes Analysis

In [ ]:
print("\nOUTCOMES (Patient-level, N={:,})".format(len(patient_level)))
print("=" * 60)

# Mortality
print(f"\nMortality:")
print(f"  Hospital mortality: {patient_level['hospital_expire_flag'].sum():,} ({patient_level['hospital_expire_flag'].mean()*100:.1f}%)")
print(f"  30-day mortality: {patient_level['mortality_30day'].sum():,} ({patient_level['mortality_30day'].mean()*100:.1f}%)")

# Length of stay
print(f"\nICU Length of Stay (days):")
print(f"  Median [IQR]: {patient_level['icu_los_days'].median():.1f} [{patient_level['icu_los_days'].quantile(0.25):.1f}-{patient_level['icu_los_days'].quantile(0.75):.1f}]")
print(f"  Mean (SD): {patient_level['icu_los_days'].mean():.1f} ({patient_level['icu_los_days'].std():.1f})")

print(f"\nHospital Length of Stay (days):")
print(f"  Median [IQR]: {patient_level['hosp_los_days'].median():.1f} [{patient_level['hosp_los_days'].quantile(0.25):.1f}-{patient_level['hosp_los_days'].quantile(0.75):.1f}]")
print(f"  Mean (SD): {patient_level['hosp_los_days'].mean():.1f} ({patient_level['hosp_los_days'].std():.1f})")

## 6. Stratified Analysis: Outcomes by Medication Use

In [ ]:
print("\nOUTCOMES STRATIFIED BY ANTIARRHYTHMIC USE")
print("=" * 60)

# Compare outcomes by AA use
aa_yes = patient_level[patient_level['received_antiarrhythmic'] == True]
aa_no = patient_level[patient_level['received_antiarrhythmic'] == False]

print(f"\nReceived Antiarrhythmic (N={len(aa_yes):,}):")
print(f"  Hospital mortality: {aa_yes['hospital_expire_flag'].mean()*100:.1f}%")
print(f"  30-day mortality: {aa_yes['mortality_30day'].mean()*100:.1f}%")
print(f"  ICU LOS (median): {aa_yes['icu_los_days'].median():.1f} days")

print(f"\nDid NOT receive Antiarrhythmic (N={len(aa_no):,}):")
print(f"  Hospital mortality: {aa_no['hospital_expire_flag'].mean()*100:.1f}%")
print(f"  30-day mortality: {aa_no['mortality_30day'].mean()*100:.1f}%")
print(f"  ICU LOS (median): {aa_no['icu_los_days'].median():.1f} days")

# Statistical tests
print(f"\nStatistical Comparison:")
chi2_mort, p_mort = stats.chi2_contingency(pd.crosstab(patient_level['received_antiarrhythmic'], 
                                                        patient_level['hospital_expire_flag']))[:2]
print(f"  Hospital mortality p-value: {p_mort:.4f}")

u_stat, p_los = stats.mannwhitneyu(aa_yes['icu_los_days'].dropna(), aa_no['icu_los_days'].dropna())
print(f"  ICU LOS p-value (Mann-Whitney): {p_los:.4f}")

## 7. Export Summary Statistics

In [ ]:
# Create summary statistics DataFrame
summary_stats = pd.DataFrame({
    'Metric': [
        'Total AF episodes',
        'Unique patients',
        'Median age (years)',
        'Female (%)',
        'Median AF duration (hours)',
        'Received antiarrhythmic (%)',
        'Received rate control (%)',
        'Hospital mortality (%)',
        '30-day mortality (%)',
        'Median ICU LOS (days)',
        'Median Hospital LOS (days)'
    ],
    'Value': [
        len(af_cohort),
        len(patient_level),
        patient_level['age'].median(),
        (patient_level['gender'] == 'F').mean() * 100,
        af_cohort['af_hours'].median(),
        patient_level['received_antiarrhythmic'].mean() * 100,
        patient_level['received_rate_control'].mean() * 100,
        patient_level['hospital_expire_flag'].mean() * 100,
        patient_level['mortality_30day'].mean() * 100,
        patient_level['icu_los_days'].median(),
        patient_level['hosp_los_days'].median()
    ]
})

summary_stats.to_csv(f'../{COHORT_OUTPUT_PATH}/summary_statistics.csv', index=False)
print("\nSummary statistics saved to summary_statistics.csv")
summary_stats

## Summary

This notebook has generated comprehensive statistical summaries of the AF cohort. Key findings are saved to CSV files.

Next: Proceed to `04_visualization_dashboard.ipynb` for visualizations.